In [1]:
import mne
import sys
sys.path.append('C:/Users/s.velut/Documents/These/Protheus_PHD/Scripts')
from Wavelets.Green_files.green.data_utils import EpochsDataset
from Wavelets.Green_files.green.wavelet_layers import RealCovariance, PW_PLV
import torch

from Wavelets.Green_files.research_code.pl_utils import get_green, GreenClassifierLM
from Wavelets.Green_files.research_code.crossval_utils import pl_crossval
from Wavelets.Green_files.tests.conftest import make_one_dummy_epoch

mne.set_log_level('ERROR')

In [2]:
from utils import get_BVEP_data, prepare_data, balance
from _utils import make_preds_accumul_aggresive
sys.path.append('C:/Users/s.velut/Documents/These/Protheus_PHD/Scripts/Wavelets/Jade/')
sys.path.insert(0,"C:\\Users\\s.velut\\Documents\\These\\moabb\\moabb\\datasets")
sys.path.insert(0,"C:\\Users\\s.velut\\Documents\\These\\moabb\\moabb\\paradigms")
import numpy as np
import time
from pyriemann.estimation import Xdawn
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from SPDNet.torch.optimizers import riemannian_adam as torch_riemannian_adam
from torch.utils.data import DataLoader, TensorDataset

from castillos2023 import CasitllosBurstVEP100

# Avec MOABB

In [5]:
# get data

subjects = [1,2,3]
# subjects = [1,2,3,4,5,6,7,8,9,10,11,12]

nb_subject = len(subjects)
moabb_ds = CasitllosBurstVEP100()

on_frame = True
recenter = False
window_size=0.5

raw_data,labels,codes,labels_codes = get_BVEP_data(subjects,on_frame,to_keep=None,moabb_ds=moabb_ds,window_size=window_size)
X_parent,Y_parent,domains_parent = prepare_data(subjects,raw_data,labels,on_frame,False,recenter,codes,window_size=window_size)
X_parent.shape

Choosing the first None classes from all possible events.


None
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
[1, 2, 3]
[1, 2, 3]
['Fp1', 'Fz', 'F3', 'F7', 'F9', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'P9', 'O1', 'Oz', 'O2', 'P10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'C4', 'Cz', 'FC2', 'FC6', 'F10', 'F8', 'F4', 'Fp2', 'stim_trial', 'stim_epoch']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'stim_trial', 'stim_epoch']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'stim_trial', 'stim_epoch']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


(3, 7020, 32, 251)

In [4]:
X_parent.shape

(3, 7020, 32, 126)

In [5]:
n = 3
n_epochs = 10
dataset = EpochsDataset(
    epochs=[make_one_dummy_epoch(n_epochs) for i in range(n)],
    targets=torch.Tensor([[0,1], [0,1],[0,1], [0,1],[0,1], [0,1],[0,1], [0,1],[0,1], [0,1]] * n).to(torch.float64),
    subjects=[f'subject_{i}' for i in range(n)],
    n_epochs=n_epochs,
)
dataset[0][0].shape

torch.Size([10, 32, 126])

In [6]:
torch.Tensor([[0,1], [0,1],[0,1], [0,1],[0,1], [0,1],[0,1], [0,1],[0,1], [0,1]] * n).to(torch.float64).shape

torch.Size([30, 2])

In [6]:
n = X_parent.shape[1]
Y = np.concatenate(np.array([[[0,1] if (y >= 0.5) else [1,0] for y in Y_parent[j]] for j in range(len(subjects))]))

dataset = EpochsDataset(
    epochs=mne.EpochsArray(np.concatenate(X_parent),info=mne.create_info(
                                        ch_names=[str(i) for i in range(32)],
                                        sfreq=500,
                                        ch_types='eeg')),
    targets=torch.Tensor(Y).to(torch.float64),
    subjects=subjects,
    n_epochs=n,
)
dataset

EpochsDataset
len: 21060
n_epochs/sample: 7020
num_channels/sample: 32
sampling frequency: 500.0
epoch duration (s): 0.5
padding: repeat
shuffle: False
random_state: Generator(PCG64)
use age: None

In [8]:
Y.shape

(21060, 2)

In [9]:
# model = get_green(
#     n_freqs=2,
#     kernel_width_s=.5,
#     n_ch=3,
#     sfreq=500,
#     orth_weights=True,
#     dropout=.5,
#     hidden_dim=[8],
#     logref='logeuclid',
#     pool_layer=PW_PLV(),
#     bi_out=[2],
#     dtype=torch.float32,
#     use_age=False,
#     out_dim=2
# )
# model_pl = GreenClassifierLM(model=model,)
# model_pl

In [7]:
model = get_green(
    n_freqs=4,
    kernel_width_s=.25,
    n_ch=32,
    sfreq=500,
    orth_weights=False,
    dropout=.5,
    hidden_dim=[8],
    logref='logeuclid',
    pool_layer=PW_PLV(),
    bi_out=[4],
    dtype=torch.float32,
    out_dim=2,
    use_age=False,
)
model_pl = GreenClassifierLM(model=model,
                            criterion=torch.nn.CrossEntropyLoss(),)
model_pl

GreenClassifierLM(
  (model): Green(
    (conv_layers): Sequential(
      (0): ComplexWavelet(kernel_width_s=0.25, sfreq=500, n_wavelets=4, stride=5, padding=0, scaling=oct)
    )
    (pooling_layers): PW_PLV()
    (spd_layers): Sequential(
      (0): LedoitWold(n_freqs=4, init_shrinkage=-3.0, learnable=True)
      (1): BiMap(d_in=32, d_out=4, n_freqs=4
    )
    (proj): LogEig(ref=logeuclid, reg=0.0001, n_freqs=4, size=4
    (head): Sequential(
      (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.5, inplace=False)
      (2): Linear(in_features=64, out_features=8, bias=True)
      (3): GELU(approximate='none')
      (4): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): Dropout(p=0.5, inplace=False)
      (6): Linear(in_features=8, out_features=2, bias=True)
    )
  )
  (criterion): CrossEntropyLoss()
)

In [11]:
pl_crossval_output, _ = pl_crossval(
    model, 
    dataset=dataset,
    n_epochs=25,
    save_preds=True,
    ckpt_prefix='checkpoints/test',
    train_splits=[[0,1,2,3,4,5,6,7]],
    test_splits=[[8]],
    batch_size=64,
    pl_module=GreenClassifierLM,
    num_workers=0, 
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 6GB Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424

validation 0
shape ypred et ytrue torch.Size([1, 2]) torch.Size([1, 2])
ypred et ytrue tensor([[-0.7690,  0.2440]], device='cuda:0') tensor([[1., 0.]], device='cuda:0', dtype=torch.float64)
1.0


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
Finding best initial lr:   0%|          | 0/20 [00:00<?, ?it/s]

training


Finding best initial lr:   5%|▌         | 1/20 [00:25<08:10, 25.82s/it]

...

validation 0
shape ypred et ytrue torch.Size([1, 2]) torch.Size([1, 2])
ypred et ytrue tensor([[-0.5910,  0.1727]], device='cuda:0') tensor([[1., 0.]], device='cuda:0', dtype=torch.float64)
1.0


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


training


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many work

pred_acc =  0.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss            1.146393060684204
       test_score                   0.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
dataset.epochs

Number of events,21060
Events,1: 21060
Time range,0.000 – 0.250 s
Baseline,off


In [8]:
from moabb.evaluations import WithinSessionEvaluation, CrossSubjectEvaluation
from moabb.paradigms import CVEP
from sklearn.pipeline import make_pipeline
from braindecode import EEGClassifier
from braindecode.models import EEGNetv4
from sklearn.pipeline import make_pipeline
from skorch.callbacks import EarlyStopping, EpochScoring
from skorch.dataset import ValidSplit

from moabb.datasets import BNCI2014_001
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import MotorImagery
from moabb.utils import setup_seed

In [9]:
# Set random seed to be able to reproduce results
seed = 42
setup_seed(seed)

# Hyperparameter
LEARNING_RATE = 0.0625 * 0.01  # parameter taken from Braindecode
WEIGHT_DECAY = 0  # parameter taken from Braindecode
BATCH_SIZE = 64  # parameter taken from BrainDecode
EPOCH = 10
PATIENCE = 3
fmin = 4
fmax = 100
tmin = 0
tmax = None

# Define a Skorch classifier
clf = EEGClassifier(
    module=model,
    optimizer=torch.optim.Adam,
    optimizer__lr=LEARNING_RATE,
    batch_size=BATCH_SIZE,
    max_epochs=EPOCH,
    train_split=ValidSplit(0.2, random_state=seed),
    device="cuda",
    callbacks=[
        EarlyStopping(monitor="valid_loss", patience=PATIENCE),
        EpochScoring(
            scoring="accuracy", on_train=True, name="train_acc", lower_is_better=False
        ),
        EpochScoring(
            scoring="accuracy", on_train=False, name="valid_acc", lower_is_better=False
        ),
    ],
    verbose=1,  # Not printing the results for each epoch
)

# Create the pipelines
pipes = {}
pipes["Green"] = make_pipeline(clf)

In [10]:

paradigm = CVEP()
dataset = [CasitllosBurstVEP100()]


evaluation = WithinSessionEvaluation(
    paradigm=paradigm, datasets=dataset, suffix="examples", overwrite=False
)
results = evaluation.process(pipes)

Choosing the first None classes from all possible events.
CasitllosBurstVEP100-WithinSession:   0%|          | 0/12 [00:00<?, ?it/s]

le sujet est 1
get the data
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
start cv
No hdf5_path provided, models will not be saved.
grid search start
cv est finie
cross validate
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.3538        0.7697       0.8221        7.6826  5.0076
      2       0.5192        0.6922       0.8221      229.0211  0.5524
      3       0.7154        0.6172       0.8221      868.0827  0.5497
Stopping since valid_loss has not improved in the last 3 epochs.
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.3275        0.7706       0.1842       10.8679  0.5259
      2       0.5116        0.6938       0.1842      410.2107  0.5242
      3       0.7098        0.6135       0.1842     3756.7553  0.5267
Stopping since valid_loss has not 

CasitllosBurstVEP100-WithinSession:   8%|▊         | 1/12 [00:26<04:46, 26.04s/it]

Stopping since valid_loss has not improved in the last 3 epochs.
return res
le sujet est 2
get the data
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
start cv
No hdf5_path provided, models will not be saved.
grid search start
cv est finie
cross validate
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.3484        0.7681       0.1779       35.4138  0.8176
      2       0.5301        0.6902       0.8221      182.2675  0.5290
      3       0.7094        0.6143       0.8221      112.5674  0.5260
Stopping since valid_loss has not improved in the last 3 epochs.
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.3283        0.7697       0.8247        7.4649  0.5494
      2       0.5221        0.6906       0.1753     3602.7817  0.5276
      3       0.7237        

CasitllosBurstVEP100-WithinSession:  17%|█▋        | 2/12 [00:45<03:41, 22.14s/it]

...

CasitllosBurstVEP100-WithinSession:  83%|████████▎ | 10/12 [03:23<00:39, 19.56s/it]

le sujet est 11
get the data
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
start cv
No hdf5_path provided, models will not be saved.
grid search start
cv est finie
cross validate
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.3326        0.7685       0.8141       18.3613  0.7336
      2       0.5268        0.6881       0.8141      317.1253  0.5590
      3       0.7201        0.6160       0.8141      230.3204  0.5449
Stopping since valid_loss has not improved in the last 3 epochs.
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.3319        0.7696       0.8283        2.7670  0.5633
      2       0.5194        0.6921       0.8283       99.1806  0.5791
      3       0.7268        0.6173       0.1717     1020.5210  0.5560
Stopping since valid_loss has not

CasitllosBurstVEP100-WithinSession:  92%|█████████▏| 11/12 [03:42<00:19, 19.32s/it]

le sujet est 12
get the data
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
start cv
No hdf5_path provided, models will not be saved.
grid search start
cv est finie
cross validate
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.3388        0.7673       0.8399        6.3398  0.7213
      2       0.5239        0.6935       0.1601     1373.8409  0.5530
      3       0.7063        0.6193       0.1601     1419.7458  0.5594
Stopping since valid_loss has not improved in the last 3 epochs.
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.3471        0.7663       0.1619       25.2622  0.5692
      2       0.5261        0.6940       0.1619      764.3405  0.5834
      3       0.7089        0.6223       0.8381       53.3334  0.5585
Stopping since valid_loss has not

CasitllosBurstVEP100-WithinSession: 100%|██████████| 12/12 [04:00<00:00, 20.07s/it]


In [ ]:
results

,score,time,samples,subject,session,channels,n_sessions,dataset,pipeline
0,0.697436,7.589004,7020.0,1,0,32,1,CasitllosBurstVEP100,Green
1,0.829060,8.038203,7020.0,2,0,32,1,CasitllosBurstVEP100,Green
2,0.829060,8.169000,7020.0,3,0,32,1,CasitllosBurstVEP100,Green
3,0.829060,10.290387,7020.0,4,0,32,1,CasitllosBurstVEP100,Green
4,0.829060,9.395892,7020.0,5,0,32,1,CasitllosBurstVEP100,Green
5,0.829060,7.266255,7020.0,6,0,32,1,CasitllosBurstVEP100,Green
6,0.829060,6.603435,7020.0,7,0,32,1,CasitllosBurstVEP100,Green
7,0.697436,10.050620,7020.0,8,0,32,1,CasitllosBurstVEP100,Green
8,0.829060,10.584088,7020.0,9,0,32,1,CasitllosBurstVEP100,Green
9,0.829060,8.627160,7020.0,10,0,32,1,CasitllosBurstVEP100,Green


In [ ]:

paradigm = CVEP()
dataset = [CasitllosBurstVEP100()]

cross_evaluation = CrossSubjectEvaluation(
    paradigm=paradigm, datasets=dataset, suffix="examples", overwrite=False
)
cross_results = cross_evaluation.process(pipes)

Choosing the first None classes from all possible events.


C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\


KeyboardInterrupt: 

In [ ]:
cross_results

,score,time,samples,subject,session,channels,n_sessions,dataset,pipeline
0,0.82906,137.261841,77220.0,1,0,32,1,CasitllosBurstVEP100,Green
1,0.82906,177.490524,77220.0,2,0,32,1,CasitllosBurstVEP100,Green
2,0.82906,137.546021,77220.0,3,0,32,1,CasitllosBurstVEP100,Green
3,0.82906,144.890167,77220.0,4,0,32,1,CasitllosBurstVEP100,Green
4,0.82906,105.100517,77220.0,5,0,32,1,CasitllosBurstVEP100,Green
5,0.82906,143.653839,77220.0,6,0,32,1,CasitllosBurstVEP100,Green
6,0.82906,105.876099,77220.0,7,0,32,1,CasitllosBurstVEP100,Green
7,0.82906,101.209442,77220.0,8,0,32,1,CasitllosBurstVEP100,Green
8,0.82906,211.256851,77220.0,9,0,32,1,CasitllosBurstVEP100,Green
9,0.82906,104.882607,77220.0,10,0,32,1,CasitllosBurstVEP100,Green


# OWN

In [3]:
# get data

# subjects = [1,2,3]
subjects = [1,2,3,4,5,6,7,8,9,10,11,12]

nb_subject = len(subjects)
moabb_ds = CasitllosBurstVEP100()


on_frame = True
recenter = True
window_size=0.25

raw_data,labels,codes,labels_codes = get_BVEP_data(subjects,on_frame,to_keep=None,moabb_ds=moabb_ds,window_size=window_size)
X_parent,Y_parent,domains_parent = prepare_data(subjects,raw_data,labels,on_frame,False,recenter,codes,window_size=window_size)
X_parent.shape

Choosing the first None classes from all possible events.


None
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\


C:\Users\s.velut\Documents\These\moabb\moabb\datasets\castillos2023.py:129: RuntimeWarning: Data file name in EEG.data (P13_burst100.fdt) is incorrect, the file name must have changed on disk, using the correct file name (P6_burst100.fdt).
  raw = mne.io.read_raw_eeglab(file_path_list[0], preload=True, verbose=False)


C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
['Fp1', 'Fz', 'F3', 'F7', 'F9', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'P9', 'O1', 'Oz', 'O2', 'P10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'C4', 'Cz', 'FC2', 'FC6', 'F10', 'F8', 'F4', 'Fp2', 'stim_trial', 'stim_epoch']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passb

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


['Fp1', 'Fz', 'F3', 'F7', 'F9', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'P9', 'O1', 'Oz', 'O2', 'P10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'C4', 'Cz', 'FC2', 'FC6', 'F10', 'F8', 'F4', 'Fp2', 'stim_trial', 'stim_epoch']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Recentering the matrix
Recentering the matrix
Recentering the matrix
Recentering the matrix
Recentering the matrix
Recentering the matrix
Recentering the matrix
Recentering the matrix
Recentering the matrix
Recentering the matrix
Recentering the matrix
Recentering the matrix


(12, 7020, 32, 126)

In [4]:
model = get_green(
    n_freqs=20,
    kernel_width_s=window_size,
    n_ch=32,
    sfreq=500,
    orth_weights=False,
    dropout=.7,
    hidden_dim=[10],
    logref='logeuclid',
    pool_layer=RealCovariance(),
    bi_out=[16,8],
    dtype=torch.float32,
    out_dim=2,
    use_age=False,
)
model_pl = GreenClassifierLM(model=model,
                            criterion=torch.nn.CrossEntropyLoss(),)
model_pl

GreenClassifierLM(
  (model): Green(
    (conv_layers): Sequential(
      (0): ComplexWavelet(kernel_width_s=0.25, sfreq=500, n_wavelets=20, stride=5, padding=0, scaling=oct)
    )
    (pooling_layers): RealCovariance()
    (spd_layers): Sequential(
      (0): LedoitWold(n_freqs=20, init_shrinkage=-3.0, learnable=True)
      (1): BiMap(d_in=32, d_out=16, n_freqs=20
      (2): BiMap(d_in=16, d_out=8, n_freqs=20
    )
    (proj): LogEig(ref=logeuclid, reg=0.0001, n_freqs=20, size=8
    (head): Sequential(
      (0): BatchNorm1d(720, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.7, inplace=False)
      (2): Linear(in_features=720, out_features=10, bias=True)
      (3): GELU(approximate='none')
      (4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): Dropout(p=0.7, inplace=False)
      (6): Linear(in_features=10, out_features=2, bias=True)
    )
  )
  (criterion): CrossEntropyLoss()
)

In [20]:
n_cal = 7
n_class = 4
nb_fold = 1
spdbn_accuracy_code_perso = np.zeros((nb_fold,nb_subject))
spdbn_tps_train_code_perso = np.zeros((nb_fold,nb_subject))
spdbn_tps_test_code_perso = np.zeros((nb_fold,nb_subject))
spdbn_accuracy_perso = np.zeros((nb_fold,nb_subject))

for k in range(nb_fold):
    for i in range(nb_subject):
        print("TL to the participant : ", i)
        X = X_parent.copy()
        Y = Y_parent.copy()
        domains = domains_parent.copy()

        ## DA
        # n_cal = 4
        # nb_sample_cal = int(n_class*n_cal*(2.2-window_size)*60)
        # ind2take = [j for j in range(len(subjects)) if j!=i]
        # X_train = np.concatenate([np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1]),X[i][:nb_sample_cal]]).reshape(-1,X.shape[-2],X.shape[-1])
        # Y_train = np.concatenate([np.concatenate(Y[ind2take]).reshape(-1),Y[i][:nb_sample_cal]]).reshape(-1)
        # X_test = X[i][nb_sample_cal:]
        # Y_test = Y[i][nb_sample_cal:]
        # labels_code_test = labels_codes[i][(n_class*4):]
        # domains_train = np.concatenate([np.concatenate(domains[ind2take]).reshape(-1),domains[i][:nb_sample_cal]]).reshape(-1)

        ## DG
        ind2take = [j for j in range(len(subjects)) if j!=i]
        X_train = np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1])
        Y_train = np.concatenate(Y[ind2take]).reshape(-1)
        X_test = X[i]
        Y_test = Y[i]
        labels_code_test = labels_codes[i]
        domains_train = np.concatenate(domains[ind2take]).reshape(-1)

        ## SS
        # nb_sample_cal = int(n_class*n_cal*(2.2-window_size)*60)
        # X_train = X[i][:nb_sample_cal]
        # Y_train = Y[i][:nb_sample_cal]
        # X_test = X[i][nb_sample_cal:]
        # Y_test = Y[i][nb_sample_cal:]
        # labels_code_test = labels_codes[i][(n_class*n_cal):]
        # domains_train = domains[i][:nb_sample_cal]
        
        # X_std = X_train.std(axis=0)
        # X_train /= X_std + 1e-8
        # X_std = X_test.std(axis=0)
        # X_test /= X_std + 1e-8

        xdawn = Xdawn(nfilter=16,classes=[1],estimator='lwf')
        xdawn = xdawn.fit(X_train, Y_train)
        X_train = xdawn.transform(X_train)
        X_test = xdawn.transform(X_test)
        X_train = np.hstack([X_train,np.tile(xdawn.evokeds_[None,:,:],(X_train.shape[0],1,1))])
        X_test = np.hstack([X_test,np.tile(xdawn.evokeds_[None,:,:],(X_test.shape[0],1,1))])

        print("balancing the number of ones and zeros")
        X_train, Y_train, domains_train = balance(X_train,Y_train,domains_train)
        print(X_train.shape)
        print(Y_train.shape)
        print(X_test.shape)

        print("Creating the different pipelines")
        lr = 1e-3
        # optimizer = riemannian_adam.RiemannianAdam(learning_rate=lr)
        batchsize = 64 #128 # 64 for burst
        clf = model

        print("Fitting")
        start = time.time()
        weight_decay = 1e-4
        
        x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42, shuffle=True)

        # Convert data into PyTorch tensors
        X_train_tensor = torch.tensor(x_train, dtype=torch.float64)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        X_val_tensor = torch.tensor(x_val, dtype=torch.float64)
        y_val_tensor = torch.tensor(y_val, dtype=torch.long)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float64)
        y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

        # Create DataLoader for train, validation, and test sets
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_dataloader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True)
        val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
        val_dataloader = DataLoader(val_dataset, batch_size=batchsize, shuffle=False)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        test_dataloader = DataLoader(test_dataset, batch_size=batchsize, shuffle=False)

        # Define loss function and optimizer
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch_riemannian_adam.RiemannianAdam(clf.parameters(), lr=lr)

        # Train the model
        num_epochs = 20

        for epoch in range(num_epochs):
            running_loss = 0.0
            train_y_pred= []
            y_train = []
            for inputs, labels in train_dataloader:
                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                # print(inputs.shape)
                # print(labels.shape)
                outputs = clf(inputs)
                # print(outputs.get_device())
                # print(labels.get_device())

                labels = labels.to('cpu')
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                optimizer.step()
                _, predicted = torch.max(outputs, 1)
                train_y_pred.append(predicted.to('cpu'))
                y_train.append(labels.to('cpu'))

                running_loss += loss.item()

            print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_dataloader)}")

        print("Training finished!")
        print("Training accuracy :", balanced_accuracy_score(np.concatenate(y_train),np.array([1 if (y >= 0.5) else 0 for y in np.concatenate(train_y_pred)])))
        spdbn_tps_train_code_perso[k][i] = time.time() - start

        # Validation
        clf.eval()
        val_correct = 0
        val_y_pred = []
        with torch.no_grad():
            for inputs, labels in val_dataloader:
                outputs = clf(inputs)
                _, predicted = torch.max(outputs, 1)
                predicted = predicted.to('cpu')
                val_correct += (predicted == labels).sum().item()
                val_y_pred.append(predicted)


        val_accuracy = balanced_accuracy_score(y_val,np.array([1 if (y >= 0.5) else 0 for y in np.concatenate(val_y_pred)]))#val_correct / len(x_val)
        print(f"Validation Accuracy: {val_accuracy}")

        # Testing
        start = time.time()
        test_correct = 0
        y_pred= []
        with torch.no_grad():
            for inputs, labels in test_dataloader:
                outputs = clf(inputs)
                _, predicted = torch.max(outputs, 1)
                predicted = predicted.to('cpu')
                y_pred.append(predicted)
                test_correct += (predicted == labels).sum().item()
                
        test_accuracy = test_correct / len(X_test)
        
        # print("getting accuracy of participant ", i)
        test_y_pred = np.concatenate(y_pred)


        # # train 
        # fitted_clf = clf.fit(x_train,y_train)
        # train_y_pred = fitted_clf.predict(x_train)
        # y_trainpred_norm = np.array([1 if (y >= 0.5) else 0 for y in train_y_pred])
        # y_traintest_norm = np.array([0 if y == 0 else 1 for y in y_train])
        # print("loss:",fitted_clf.get_loss(y_trainpred_norm,y_traintest_norm))
        # print("train accuracy", balanced_accuracy_score(y_traintest_norm,y_trainpred_norm))

        # # validate
        # val_y_pred = fitted_clf.predict(x_val)
        # y_valpred_norm = np.array([1 if (y >= 0.5) else 0 for y in val_y_pred])
        # y_valtest_norm = np.array([0 if y == 0 else 1 for y in y_val])
        # print("val accuracy", balanced_accuracy_score(y_valtest_norm,y_valpred_norm))


        # # y_pred = np.array(y_pred)
        # test_y_pred = fitted_clf.pred(X_test)

        y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in test_y_pred])
        y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])

        # tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
        spdbn_accuracy_perso[k][i] = balanced_accuracy_score(y_test_norm,y_pred_norm)
        print(f"Test Accuracy: {spdbn_accuracy_perso[k][i]}")

        labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred_norm, codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
        )
        spdbn_tps_test_code_perso[k][i] = time.time() - start
        spdbn_accuracy_code_perso[k][i] = np.round(balanced_accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
        # keras.backend.clear_session()


print(spdbn_accuracy_perso)
print(spdbn_tps_train_code_perso)
print(spdbn_tps_test_code_perso)
print(spdbn_accuracy_code_perso)

spdbn_accuracy_perso = np.mean(spdbn_accuracy_perso,axis=0)
spdbn_tps_train_code_perso = np.mean(spdbn_tps_train_code_perso,axis=0)
spdbn_tps_test_code_perso = np.mean(spdbn_tps_test_code_perso,axis=0)
spdbn_accuracy_code_perso = np.mean(spdbn_accuracy_code_perso,axis=0)

print(spdbn_accuracy_perso)
print(spdbn_tps_train_code_perso)
print(spdbn_tps_test_code_perso)
print(spdbn_accuracy_code_perso)
# np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDbnNet/score/WO1_score",spdbn_accuracy_perso)
# np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDbnNet/score_code/WO1_score_code",spdbn_accuracy_code_perso)
# np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDbnNet/temps_train_code/WO1_tps_train_code",spdbn_tps_train_code_perso)
# np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDbnNet/temps_test_code/WO1_tps_test_code",spdbn_tps_test_code_perso)

TL to the participant :  0
balancing the number of ones and zeros
(26400, 32, 126)
(26400,)
(7020, 32, 126)
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.6374088458039544
Epoch 2, Loss: 0.5308170996832126
Epoch 3, Loss: 0.5198487840818636
Epoch 4, Loss: 0.5161755153627107
Epoch 5, Loss: 0.5105622343944781
Epoch 6, Loss: 0.5075623751589746
Epoch 7, Loss: 0.501397329568863
Epoch 8, Loss: 0.4986560123436379
Epoch 9, Loss: 0.49700562114065344
Epoch 10, Loss: 0.4998178710540136
Epoch 11, Loss: 0.49438112170407267
Epoch 12, Loss: 0.4925415687488787
Epoch 13, Loss: 0.4919468501300523
Epoch 14, Loss: 0.4914361037088163
Epoch 15, Loss: 0.48760880042206156
Epoch 16, Loss: 0.4858998176726428
Epoch 17, Loss: 0.4845502357591282
Epoch 18, Loss: 0.48732415970527765
Epoch 19, Loss: 0.48557239093563775
Epoch 20, Loss: 0.48463608983791234
Training finished!
Training accuracy : 0.7836994152412093
Validation Accuracy: 0.7916214137172075
Test Accuracy: 0.8207946735395188
TL to the participant :

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  2
balancing the number of ones and zeros
(26400, 32, 126)
(26400,)
(7020, 32, 126)
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.4822502545335076
Epoch 2, Loss: 0.42163950990546833
Epoch 3, Loss: 0.4026604581962932
Epoch 4, Loss: 0.38708931243780886
Epoch 5, Loss: 0.37544187022881076
Epoch 6, Loss: 0.36281825335639895
Epoch 7, Loss: 0.3544424905921474
Epoch 8, Loss: 0.3407100919521216
Epoch 9, Loss: 0.3298031236186172
Epoch 10, Loss: 0.32185350647478395
Epoch 11, Loss: 0.3102333204764308
Epoch 12, Loss: 0.3016663381547639
Epoch 13, Loss: 0.2962322555708163
Epoch 14, Loss: 0.28460321047089315
Epoch 15, Loss: 0.27961734998406784
Epoch 16, Loss: 0.2688489294413364
Epoch 17, Loss: 0.2612505653590867
Epoch 18, Loss: 0.25705774970578427
Epoch 19, Loss: 0.24621955458865022
Epoch 20, Loss: 0.23939033469015902
Training finished!
Training accuracy : 0.8956405005229193
Validation Accuracy: 0.7956265805757331
Test Accuracy: 0.7810524054982818
TL to the particip

In [16]:
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/GREEN/score/DG_score",spdbn_accuracy_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/GREEN/score_code/DG_score_code",spdbn_accuracy_code_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/GREEN/temps_train_code/DG_tps_train_code",spdbn_tps_train_code_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/GREEN/temps_test_code/DG_tps_test_code",spdbn_tps_test_code_perso)